In [45]:
import pandas as pd
import numpy as np

In [9]:
df1 = pd.read_excel('source/21852_P960944_Traffic_Stops_2022_YTD.xlsx', sheet_name='TRAFFIC STOPS 1')

In [2]:
df2 = pd.read_excel('source/21852_P960944_Traffic_Stops_2022_YTD.xlsx', sheet_name='TRAFFIC STOPS 2')

In [12]:
df_og = pd.concat([df1, df2], ignore_index=True)

In [13]:
# make a working copy
df = df_og.copy()

In [11]:
df.head()

,DATE,DATE/TIME OF STOP,STOP BLOCK-LEVEL ADDRESS,LOCATION BEAT,CPD UNIT NO,YOB,RACE,SEX,TICKET ISSUED Y/N,INITIAL STATUTE,...,VEH OTHER AMT,DRV/PASS DRUGS AMT,DRV/PASS WEAP AMT,DRV/PASS ALCOHOL AMT,DRV/PASS STOLEN PROP AMT,DRV/PASS DRUG PARAH AMT,DRV/PASS OTHER,DRV/PASS OTHER AMT,CONTACT CARD NO,month_year
0,22870227,01/01/2022 00:01,86XX S WOLCOTT AVE,614,6.0,1958,BLACK,M,N,"9-76-050(D) LIGHT, LICENSE PLATE",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21771149,01/01/2022 00:03,56XX W BELMONT AVE,2514,16.0,1964,WHITE,M,N,9-36-020(B) OVERTAKING VEHICLE ON RIGHT-DRIVIN...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22882362,01/01/2022 00:09,2XX S WOOD ST,1225,213.0,1979,BLACK,M,N,9-76-050(B) HEADLIGHT TWO REQUIRED-MOTOR VEHICLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21779781,01/01/2022 00:24,18XX W PETERSON AVE,2413,24.0,1983,BLACK,M,N,9-80-020 RED LIGHTS & FLASHING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23019248,01/01/2022 00:30,46XX W PALMER ST,2522,25.0,1991,HISPANIC,F,N,"9-76-050(D) LIGHT, LICENSE PLATE",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Process data

In [14]:
df['DATE/TIME OF STOP'] = pd.to_datetime(df['DATE/TIME OF STOP'])

In [31]:
df['day'] = df['DATE/TIME OF STOP'].dt.strftime('%Y-%m-%d')

In [ ]:
df['year'] = df['DATE/TIME OF STOP'].dt.strftime('%Y-%m-%d')

In [ ]:
# create subsets

In [57]:
conditions = [
    (df['day']>='2023-01-01') & (df['day']<='2023-06-30'),
    (df['day']>='2022-01-01') & (df['day']<='2022-06-30'),
    (df['day']>='2024-01-01') & (df['day']<='2024-06-30')]
choices = ['1st_half_2023', '1st_half_2022', '1st_half_2024']
df['period'] = np.select(conditions, choices, default='')

In [59]:
df.groupby('period').size()

period
                 514195
1st_half_2022    246963
1st_half_2023    295110
1st_half_2024    177342
dtype: int64

# Total stops by month

In [16]:
df.groupby('month_year').size().reset_index(name='count')

,month_year,count
0,01/2022,34125
1,01/2023,60379
2,01/2024,46628
3,02/2022,37665
4,02/2023,54203
5,02/2024,35144
6,03/2022,40166
7,03/2023,51886
8,03/2024,32445
9,04/2022,44027


Stops declined in 2024.

In [CPD stops by month](https://docs.google.com/spreadsheets/d/1npZSD9v7Fjfw0vAodHsmfOL6pgSq3V6fiIzNOjGhmx8/edit?usp=sharing)

# Stops by race by month

In [17]:
# 145 stops missing race
df.groupby('RACE', dropna=False).size()

RACE
AMER IND/ALASKAN NATIVE             12
AMER INDIAN / ALASKAN NATIVE      6732
ASIAN                            45545
BLACK                           650793
HAWAIIAN/PACIFIC ISLANDER         5165
HISPANIC                        356272
WHITE                           168946
NaN                                145
dtype: int64

In [19]:
pd.pivot_table(df,
              index='month_year',
              columns='RACE',
              values='DATE/TIME OF STOP', # every record has date/time
              aggfunc='count')

RACE,AMER IND/ALASKAN NATIVE,AMER INDIAN / ALASKAN NATIVE,ASIAN,BLACK,HAWAIIAN/PACIFIC ISLANDER,HISPANIC,WHITE
month_year,,,,,,,
01/2022,NaN,225.0,1169.0,19263.0,114.0,8347.0,5003.0
01/2023,NaN,332.0,2275.0,32175.0,229.0,17209.0,8154.0
01/2024,NaN,283.0,2012.0,21072.0,123.0,16038.0,7099.0
02/2022,NaN,186.0,1262.0,21866.0,91.0,9114.0,5142.0
02/2023,NaN,265.0,2104.0,28898.0,294.0,15048.0,7588.0
02/2024,NaN,201.0,1480.0,16076.0,96.0,11729.0,5561.0
03/2022,NaN,194.0,1258.0,23637.0,169.0,9539.0,5367.0
03/2023,NaN,257.0,1962.0,27587.0,280.0,14373.0,7418.0
03/2024,NaN,208.0,1495.0,14406.0,92.0,11382.0,4861.0


Compare racial breakdown of stops in the first 6 months of 2024 with previous years

In [60]:
pd.pivot_table(df,
              index='RACE',
              columns='period',
              values='DATE/TIME OF STOP',
              aggfunc='count')

period,,1st_half_2022,1st_half_2023,1st_half_2024
RACE,,,,
AMER IND/ALASKAN NATIVE,12.0,NaN,NaN,NaN
AMER INDIAN / ALASKAN NATIVE,2848.0,1245.0,1554.0,1085.0
ASIAN,18724.0,7824.0,10740.0,8257.0
BLACK,270476.0,143883.0,156562.0,79872.0
HAWAIIAN/PACIFIC ISLANDER,2395.0,932.0,1334.0,504.0
HISPANIC,150674.0,60549.0,84678.0,60371.0
WHITE,68980.0,32507.0,40210.0,27249.0


In [41]:
g = df24_1st.groupby('RACE').size().reset_index(name='stops')
g['pct'] = g['stops']/g['stops'].sum()
g

,RACE,stops,pct
0,AMER INDIAN / ALASKAN NATIVE,1085,0.006118
1,ASIAN,8257,0.046561
2,BLACK,79872,0.450394
3,HAWAIIAN/PACIFIC ISLANDER,504,0.002842
4,HISPANIC,60371,0.340429
5,WHITE,27249,0.153656


In [42]:
g = df23_1st.groupby('RACE').size().reset_index(name='stops')
g['pct'] = g['stops']/g['stops'].sum()
g

,RACE,stops,pct
0,AMER INDIAN / ALASKAN NATIVE,1554,0.005266
1,ASIAN,10740,0.036397
2,BLACK,156562,0.530578
3,HAWAIIAN/PACIFIC ISLANDER,1334,0.004521
4,HISPANIC,84678,0.286968
5,WHITE,40210,0.136269


In [43]:
g = df22_1st.groupby('RACE').size().reset_index(name='stops')
g['pct'] = g['stops']/g['stops'].sum()
g

,RACE,stops,pct
0,AMER INDIAN / ALASKAN NATIVE,1245,0.005042
1,ASIAN,7824,0.031684
2,BLACK,143883,0.582664
3,HAWAIIAN/PACIFIC ISLANDER,932,0.003774
4,HISPANIC,60549,0.245197
5,WHITE,32507,0.131639


The share of Black drivers stopped has declined by 13% from 58% in the first half of 2022 to 45% in the first half of 2024. Black drivers did not make up the majority of drivers stopped by CPD in the first half 2024. <br>

<b>TODO:</b> Is it the first time since 2004 that Black drivers did not make up the majority of CPD stops?

# Stops by beat

In [62]:
beat_pivot = pd.pivot_table(df,
              index='LOCATION BEAT',
              columns='period',
              values='DATE/TIME OF STOP',
              aggfunc='count')

In [66]:
beat_pivot.reset_index(inplace=True)

In [67]:
beat_pivot

period,LOCATION BEAT,,1st_half_2022,1st_half_2023,1st_half_2024
0,111,1602.0,884.0,1096.0,440.0
1,112,1218.0,747.0,886.0,495.0
2,113,1624.0,767.0,912.0,1130.0
3,114,2141.0,1331.0,1282.0,733.0
4,121,444.0,264.0,201.0,215.0
...,...,...,...,...,...
272,2533,3391.0,1215.0,1802.0,2246.0
273,2534,2469.0,985.0,1430.0,1276.0
274,2535,2398.0,1117.0,1447.0,1167.0
275,3100,2637.0,1345.0,1600.0,1350.0


In [75]:
len(df[df['period'] == '1st_half_2024']) - len(df[df['period'] == '1st_half_2023'])

-117768

In [77]:
(len(df[df['period'] == '1st_half_2024']) - len(df[df['period'] == '1st_half_2023']))/len(df[df['period'] == '1st_half_2023'])

-0.39906475551489273

In [70]:
# export for flourish
beat_pivot.to_csv('output/first_half_by_beat.csv', index=False)

Comparing the first six months of 2024 with the first six months of 2023, the overall number of stops citywide declined by 117,768, or 40%. But the decline wasn't even throughout the city. Stops declined in nearly every beat except for most beats in the 25th and 14th districts on the Northwest Side.

[Flourish viz](https://public.flourish.studio/visualisation/18743732/) <br>
In [CPD Jan thru Jun stops by beat 2022-2024](https://docs.google.com/spreadsheets/d/1m4uKnwavTyTRuGq1hvJF04ZciVCPSVIqIZYl3D_xshs/edit?usp=sharing)

<b>TODO:</b> Add district to beat crosswalk

# Arrests